# COGS 108 - Data Checkpoint

# Names

- Aditya Sriram
- Nicole Liu
- Weston Chester
- Sophia Conti
- Katherine Gao

<a id='research_question'></a>
# Research Question

Is California on track to reach its goal of 100% zero emission sales by 2035?

Hypothesis: Based on previous data, we believe that California is on track to meet its 100% zero emission sales goal by 2035.

# Dataset(s)

- Dataset Name: New_ZEV_Sales_Last_updated_04-21-2023_ada 
- Link to the dataset: https://www.energy.ca.gov/files/zev-and-infrastructure-stats-data
- Number of observations: 18136

This dataset is a csv file containing information about the number of ZEVs (zero emission vehicles) sold per year. The data contains columns variables such as year, fuel type, county of sale, make and model of car, and total number of vehicles sold per year. 

- Dataset Name: Vehicles Registered By County 
- Link to the dataset: https://www.dmv.ca.gov/portal/dmv-research-reports/research-development-data-dashboards/vehicles-registered-by-county/
- Number of observations: 882 

This dataset is a collectio of csv files that list the number of cars registered per county in California. The set contains variables such as county, autos, trucks, trailers, motorcycles, and the total per county.

In order to use this dataset along side the next, we plan on grouping by county and then merging the two sets together. 

# Setup

In [37]:
import pandas as pd

zev_sales = pd.read_csv('Data/zev_sales.csv')

reg_2008 = pd.read_csv('Data/Vehicles Registered By County-2008.csv', encoding='utf-16')
reg_2009 = pd.read_csv('Data/Vehicles Registered By County-2009.csv', encoding='utf-16')
reg_2010 = pd.read_csv('Data/Vehicles Registered By County-2010.csv', encoding='utf-16')
reg_2011 = pd.read_csv('Data/Vehicles Registered By County-2011.csv', encoding='utf-16')
reg_2012 = pd.read_csv('Data/Vehicles Registered By County-2012.csv', encoding='utf-16')
reg_2013 = pd.read_csv('Data/Vehicles Registered By County-2013.csv', encoding='utf-16')
reg_2014 = pd.read_csv('Data/Vehicles Registered By County-2014.csv', encoding='utf-16')
reg_2015 = pd.read_csv('Data/Vehicles Registered By County-2015.csv', encoding='utf-16')
reg_2016 = pd.read_csv('Data/Vehicles Registered By County-2016.csv', encoding='utf-16')
reg_2017 = pd.read_csv('Data/Vehicles Registered By County-2017.csv', encoding='utf-16')
reg_2018 = pd.read_csv('Data/Vehicles Registered By County-2018.csv', encoding='utf-16')
reg_2019 = pd.read_csv('Data/Vehicles Registered By County-2019.csv', encoding='utf-16')
reg_2020 = pd.read_csv('Data/Vehicles Registered By County-2020.csv', encoding='utf-16')
reg_2021 = pd.read_csv('Data/Vehicles Registered By County-2021.csv', encoding='utf-16')

# Data Cleaning

We start with the dataframe for ZEV sales.

In [2]:
zev_sales.isna().groupby('Data Year').sum()

,County,FUEL_TYPE,MAKE,MODEL,Number of Vehicles
Data Year,,,,,
False,0,0,0,0,0


Looks like theres no NaN data in this set. We should check for placeholder values ('-' or '-99'), though.

In [3]:
zev_sales[zev_sales.eq('-').any(1)].shape[0] + zev_sales[zev_sales.eq('-99').any(1)].shape[0]

TypeError: any() takes 1 positional argument but 2 were given

Looks like theres no '-' or '-99' either! Good job DMV.

In either case, we are looking to merge this dataset with a secondary one about car registrations, which only has data up to the year 2021. We should filter this dataset (which is about number of cars sold) to the time period of cars sold in 2021 and before. 

In [ ]:
zev_sales = zev_sales[zev_sales['Data Year'] <= 2021]

zev_sales

We also see that the age of the average car in 2021 is [12 years](https://www.caranddriver.com/news/a33457915/average-age-vehicles-on-road-12-years/). However, EV batteries are predicted to last between [12-15 years](https://cars.usnews.com/cars-trucks/advice/how-long-do-ev-batteries-last), meaning that they would potentially last longer than the body of the car itself. Thus, we will say that we expect the average lifetime of an electric vehicle to be 12 years. Our dataset has its oldest car dating to 1998. By the average 12 year lifespan, we no longer expect that car to be in use and so it should be removed from the dataset. <br>

If we look at total automobile registrations in 2021, which is the most recent year from the other dataset, we would expect the oldest car registered (on average) to be 12 years old. This leads us to the conclusion that we should only look at ZEVs sold in or after 2009 in order to remain in the proper time frame.

In [ ]:
zev_sales = zev_sales[zev_sales['Data Year'] >= 2009]

zev_sales

Now onto the total registrations dataset. Some intial cleaning was done to the data by hand as the csv data was delimited by tabs instead of commas, so the delimiter were replaced inorder to be read in. 

In [46]:
all_sets = [reg_2008,reg_2009,reg_2010,reg_2011,reg_2012,reg_2013,reg_2014,reg_2015,reg_2016,reg_2017,reg_2018,reg_2019,reg_2020,reg_2021]
year = 2008
for df in all_sets:
    df['Year'] = year
    year += 1;
    
registrations = pd.concat(all_sets).reset_index(drop= True)
registrations
    

,Counties,Autos,Trucks,Trailers,Motorcycles,Total Vehicles,Year
0,ALAMEDA,932092.0,194392.0,71257.0,31389.0,1229130,2008
1,ALPINE,1030.0,526.0,360.0,62.0,1978,2008
2,AMADOR,26926.0,15114.0,10163.0,2113.0,54316,2008
3,BUTTE,121017.0,54233.0,43693.0,6965.0,225908,2008
4,CALAVERAS,34297.0,19762.0,14928.0,2771.0,71758,2008
...,...,...,...,...,...,...,...
877,OUT OF STATE,109678.0,46415.0,66372.0,4975.0,227440,2021
878,IRP VEHICLES*,NaN,NaN,NaN,NaN,2099129,2021
879,MISC VEHICLES,NaN,NaN,NaN,NaN,121716,2021
880,FEE EXEMPT VEHICLES,231370.0,309182.0,75483.0,13223.0,629258,2021


We can start out by first merging the sperate tables into one and then adding their year as a variable. We can now move on to removing non relevant data.

In [47]:
registrations=registrations.dropna()
registrations= registrations[registrations.Counties != 'FEE EXEMPT VEHICLES'][registrations.Counties !='IRP VEHICLES*'][registrations.Counties !='MISC VEHICLES'][registrations.Counties !='STATEWIDE']
registrations

C:\Users\17144\AppData\Local\Temp\ipykernel_30104\135009615.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  registrations= registrations[registrations.Counties != 'FEE EXEMPT VEHICLES'][registrations.Counties !='IRP VEHICLES*'][registrations.Counties !='MISC VEHICLES'][registrations.Counties !='STATEWIDE']
C:\Users\17144\AppData\Local\Temp\ipykernel_30104\135009615.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  registrations= registrations[registrations.Counties != 'FEE EXEMPT VEHICLES'][registrations.Counties !='IRP VEHICLES*'][registrations.Counties !='MISC VEHICLES'][registrations.Counties !='STATEWIDE']
C:\Users\17144\AppData\Local\Temp\ipykernel_30104\135009615.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  registrations= registrations[registrations.Counties != 'FEE EXEMPT VEHICLES'][registrations.Counties !='IRP VEHICLES*'][registrations.Counties !='MISC VEHICLES'][regist

,Counties,Autos,Trucks,Trailers,Motorcycles,Total Vehicles,Year
0,ALAMEDA,932092.0,194392.0,71257.0,31389.0,1229130,2008
1,ALPINE,1030.0,526.0,360.0,62.0,1978,2008
2,AMADOR,26926.0,15114.0,10163.0,2113.0,54316,2008
3,BUTTE,121017.0,54233.0,43693.0,6965.0,225908,2008
4,CALAVERAS,34297.0,19762.0,14928.0,2771.0,71758,2008
...,...,...,...,...,...,...,...
873,TUOLUMNE,40557.0,22158.0,15416.0,3076.0,81207,2021
874,VENTURA,596553.0,149531.0,54714.0,24345.0,825143,2021
875,YOLO,124118.0,38217.0,26301.0,4308.0,192944,2021
876,YUBA,44088.0,17588.0,13875.0,2485.0,78036,2021


We can first drop any rows with null values. We can also remove rows that include data on nonconsumer vehicles, which includes the statewide data.

In [48]:
registrations = registrations.drop(columns=['Trailers', 'Motorcycles'])
registrations['Total Vehicles'] = registrations['Autos'] + registrations['Trucks']
registrations

,Counties,Autos,Trucks,Total Vehicles,Year
0,ALAMEDA,932092.0,194392.0,1126484.0,2008
1,ALPINE,1030.0,526.0,1556.0,2008
2,AMADOR,26926.0,15114.0,42040.0,2008
3,BUTTE,121017.0,54233.0,175250.0,2008
4,CALAVERAS,34297.0,19762.0,54059.0,2008
...,...,...,...,...,...
873,TUOLUMNE,40557.0,22158.0,62715.0,2021
874,VENTURA,596553.0,149531.0,746084.0,2021
875,YOLO,124118.0,38217.0,162335.0,2021
876,YUBA,44088.0,17588.0,61676.0,2021


We have no removed any vehicles that we are not focusing on, such as motorcycles and trailers, and have updated the total vehicles column accordingly.

In [50]:
registrations['Delta'] = 0 
registrations = registrations.sort_values(['Counties','Year']).reset_index(drop= True)
registrations['Delta'].loc[registrations['Year']!=2008] = registrations['Total Vehicles'].diff()
registrations


C:\Users\17144\AppData\Local\Temp\ipykernel_30104\2629021328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  registrations['Delta'].loc[registrations['Year']!=2008] = registrations['Total Vehicles'].diff()


,Counties,Autos,Trucks,Total Vehicles,Year,Delta
0,ALAMEDA,932092.0,194392.0,1126484.0,2008,0
1,ALAMEDA,937076.0,188715.0,1125791.0,2009,-693
2,ALAMEDA,952005.0,184257.0,1136262.0,2010,10471
3,ALAMEDA,954158.0,177376.0,1131534.0,2011,-4728
4,ALAMEDA,974615.0,175467.0,1150082.0,2012,18548
...,...,...,...,...,...,...
821,YUBA,40802.0,16102.0,56904.0,2017,777
822,YUBA,41347.0,16227.0,57574.0,2018,670
823,YUBA,42839.0,16745.0,59584.0,2019,2010
824,YUBA,42929.0,17029.0,59958.0,2020,374


We can now add a column that tells us the difference in registered cars from year to year in order to observe overall treands in car ownership. With this the data is now ready to be compared with the other dataset.